In [ ]:
print("The following is for exercise 3")

The following is for exercise 3


In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 70 kB 9.8 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629484 sha256=f3a5559d5dbb479647d5a5087afc9b6f47b2311695158534411739f680624c06
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=762226952d95f1c47d3b0794556dec2bf37a104c4e4e3bb8a3b613e4eb5b79d9
  Stored in directory: /root/.cache/pip/wheels/37/5e/9e/76d7430e116b7cab0016fbabb26b896daae1946a3f7dea9915
Successfully built pycuda pytools


In [ ]:
import numpy
numpy.random.seed(1)
a = numpy.random.randn(128)

In [ ]:
# Convert values from double precision to single precision (requirement for CUDA)
a = a.astype(numpy.float32)

In [ ]:
# Allocate memory in device
a_gpu = cuda.mem_alloc(a.nbytes)

In [ ]:
# Copy array data from a to a_gpu (from host to device)
cuda.memcpy_htod(a_gpu, a)

In [ ]:
# Create CUDA kernel
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = (blockIdx.x*blockDim.x + threadIdx.x)*2;
    
    a[idx] *= 2;
    a[idx + 1] *= 2;
  }
  """)

In [ ]:
# Here we choose the block and grid dimensions
# TODO: add grid dimensions
grid_dim = (1,1)
# TODO: add block dimensions
block_dim = (64,1,1)
func = mod.get_function("doublify")
func(a_gpu, block=block_dim, grid = grid_dim)

In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled[:10])
print(a[:10])

[ 3.2486908  -1.2235128  -1.0563436  -2.1459372   1.7308153  -4.6030774
  3.4896235  -1.5224138   0.6380782  -0.49874076]
[ 1.6243454  -0.6117564  -0.5281718  -1.0729686   0.86540765 -2.3015387
  1.7448118  -0.7612069   0.3190391  -0.24937038]
